In [1]:
#Importing libraries

import pandas as pd
import  matplotlib.pyplot as plt
import os

In [2]:
#All monthly data combined to one dataframe

dir = "Sales Data/"
files = []
df = pd.DataFrame

for file in os.listdir(dir):
    read_df = pd.read_csv(dir + file)
    files.append(read_df)

df = pd.concat(files)

df.dropna(inplace=True)
df.drop_duplicates(keep= False, inplace=True)
df.head(10)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
5,176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001"
6,176562,USB-C Charging Cable,1,11.95,04/29/19 13:03,"381 Wilson St, San Francisco, CA 94016"
7,176563,Bose SoundSport Headphones,1,99.99,04/02/19 07:46,"668 Center St, Seattle, WA 98101"
8,176564,USB-C Charging Cable,1,11.95,04/12/19 10:58,"790 Ridge St, Atlanta, GA 30301"
9,176565,Macbook Pro Laptop,1,1700,04/24/19 10:38,"915 Willow St, San Francisco, CA 94016"
10,176566,Wired Headphones,1,11.99,04/08/19 14:05,"83 7th St, Boston, MA 02215"


# What was the best month for sales? How much was earned that month?

In [3]:
#Add Month amd Year column to filter data by dates
df["Order Date"] = pd.to_datetime(df["Order Date"], errors="coerce")
df["Month"] = df["Order Date"].dt.month.astype(int)
df["Year"] = df["Order Date"].dt.year.astype(int)


In [4]:
#Add a column to get order total per Order ID

# df["Order Total"] = df["Quantity Ordered"] * float(df["Price Each"]
# df.head(10)


#Error because types are objects not numbers
df.dtypes

Order ID                    object
Product                     object
Quantity Ordered            object
Price Each                  object
Order Date          datetime64[ns]
Purchase Address            object
Month                        int32
Year                         int32
dtype: object

In [5]:
#Shortcut to remove column titles that were spread througout data
df.dropna(inplace= True)

#Change values in these columns from objects to numbers to be able to perform maths
df["Quantity Ordered"] = pd.to_numeric(df["Quantity Ordered"])
df["Price Each"] = pd.to_numeric(df["Price Each"])
df.dtypes


Order ID                    object
Product                     object
Quantity Ordered             int64
Price Each                 float64
Order Date          datetime64[ns]
Purchase Address            object
Month                        int32
Year                         int32
dtype: object

In [6]:
df["Order Total"] = df["Quantity Ordered"].astype(float) * df["Price Each"]
df.head(10)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Year,Order Total
0,176558,USB-C Charging Cable,2,11.95,2019-04-19 08:46:00,"917 1st St, Dallas, TX 75001",4,2019,23.90
2,176559,Bose SoundSport Headphones,1,99.99,2019-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215",4,2019,99.99
3,176560,Google Phone,1,600.00,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",4,2019,600.00
4,176560,Wired Headphones,1,11.99,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",4,2019,11.99
5,176561,Wired Headphones,1,11.99,2019-04-30 09:27:00,"333 8th St, Los Angeles, CA 90001",4,2019,11.99
6,176562,USB-C Charging Cable,1,11.95,2019-04-29 13:03:00,"381 Wilson St, San Francisco, CA 94016",4,2019,11.95
7,176563,Bose SoundSport Headphones,1,99.99,2019-04-02 07:46:00,"668 Center St, Seattle, WA 98101",4,2019,99.99
8,176564,USB-C Charging Cable,1,11.95,2019-04-12 10:58:00,"790 Ridge St, Atlanta, GA 30301",4,2019,11.95
9,176565,Macbook Pro Laptop,1,1700.00,2019-04-24 10:38:00,"915 Willow St, San Francisco, CA 94016",4,2019,1700.00
10,176566,Wired Headphones,1,11.99,2019-04-08 14:05:00,"83 7th St, Boston, MA 02215",4,2019,11.99


In [7]:
#Group all sales together based on the month and calculte the total amount of products ordered and their total price
monthly_totals = df.groupby(["Month"])[["Quantity Ordered", "Order Total"]].sum()
monthly_totals

,Quantity Ordered,Order Total
Month,,
1,10883,1820569.59
2,13413,2198133.74
3,16953,2802846.32
4,20514,3387765.72
5,18639,3148625.71
6,15215,2574758.04
7,16036,2645146.88
8,13410,2237698.86
9,13073,2091371.25


In [8]:
#Get the row that has the maximum about of revenue
monthly_totals[monthly_totals["Order Total"] == monthly_totals["Order Total"].max()]

,Quantity Ordered,Order Total
Month,,
12,28034,4603148.06


### December was the highest grossing month of the year with £4,603,148.06 in revenue
---

# What time should we display advertisements to maximise the likelihood of customers buying a product?

In [9]:
# Adding an hour column to filter data
df["Hour"] = df["Order Date"].dt.hour

#Group all of the sales by hour and sum up the amount of sales made per hour timeslot, then sort the values and remove irrelevant data.
hourly_sales = df.groupby("Hour").sum(numeric_only= True)
hourly_sales.sort_values("Quantity Ordered", ascending=False).head(1).drop(["Price Each", "Month", "Year"], axis=1)

,Quantity Ordered,Order Total
Hour,,
19,14432,2411003.74


In [10]:
#Repeated previous step but rather than using the amount of orders placed used the amount of revenue generated.
hourly_sales.sort_values("Order Total", ascending= False).head(1).drop(["Price Each", "Month", "Year"], axis=1)

,Quantity Ordered,Order Total
Hour,,
19,14432,2411003.74


### The timeslot where most orders are place fall between 7pm -8pm. Advertising should be scheduled to take advantage of this time slot.
---

# What products are most often sold together?

In [11]:
#Joined information from the products column that shared an Order ID into a single column
df['Grouped'] = df.groupby('Order ID')['Product'].transform(lambda x: ','.join(x))

In [12]:
#Create data subset of only necessary information
grouped = df[["Order ID", "Grouped"]]

#Removed all duplicate entries keeping only the first.
grouped.drop_duplicates(inplace= True)

#Created subset of data where the new column contained the , seperator and counted how many order numbers are attached to it.
orders = grouped[grouped["Grouped"].str.contains(",")]
orders.groupby("Grouped")["Grouped"].agg('count').to_frame("Count").reset_index().sort_values("Count", ascending= False)

C:\Users\Ryan\AppData\Local\Temp\ipykernel_14992\4054720712.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grouped.drop_duplicates(inplace= True)


,Grouped,Count
329,"iPhone,Lightning Charging Cable",882
165,"Google Phone,USB-C Charging Cable",857
348,"iPhone,Wired Headphones",361
286,"Vareebadd Phone,USB-C Charging Cable",312
180,"Google Phone,Wired Headphones",304
...,...,...
182,"Google Phone,Wired Headphones,Macbook Pro Laptop",1
183,"Google Phone,Wired Headphones,USB-C Charging C...",1
284,"Vareebadd Phone,Lightning Charging Cable",1
185,"LG Dryer,27in 4K Gaming Monitor",1


### An Iphone and a Lightning Charging Cable are the most commonly bought together items
---

# What is the most commonly bought item?

In [13]:
df.groupby("Product")["Order ID"].agg("count").to_frame("Count").reset_index().sort_values("Count", ascending= False)

,Product,Count
15,USB-C Charging Cable,21815
12,Lightning Charging Cable,21562
5,AAA Batteries (4-pack),20583
4,AA Batteries (4-pack),20539
17,Wired Headphones,18816
6,Apple Airpods Headphones,15501
7,Bose SoundSport Headphones,13271
2,27in FHD Monitor,7489
18,iPhone,6838
1,27in 4K Gaming Monitor,6220


### USB-C Charging Cable is the most bought item
---